In [3]:
# standard libs
import os
import sys
import logging

# project lib
PROJECT_SRC_PATH = os.path.join(os.path.abspath(''), '..', 'src')
sys.path.append(PROJECT_SRC_PATH)

import utils
import dataset
import visualizations
from prediction_age import AgePredictor, AgeClassifier, AgePredictorComparison
from preprocessing import *
from fragmented_cities import *

# external libs
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely import wkt

import shap

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from xgboost import XGBRegressor, XGBClassifier, XGBRFClassifier

/usr/local/Caskroom/miniconda/base/envs/uf-ml/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import warnings
warnings.filterwarnings('ignore', message='pandas.Int64Index is deprecated')
logging.captureWarnings(True)

In [4]:
%load_ext autoreload
%autoreload 2

## Data

In [5]:
overview_france = pd.read_csv(os.path.join('..', 'data', 'france_overview.csv'))
overview_netherlands = pd.read_csv(os.path.join('..', 'data', 'netherlands_overview.csv'))
overview_spain = pd.read_csv(os.path.join('..', 'data', 'spain_overview.csv'))
overview_finland = pd.read_csv(os.path.join('..', 'data', 'finland_overview.csv'))


In [24]:
gadm_france = gpd.read_file(os.path.join('..', 'data', 'gadm', 'gadm36_FRA_4.shp'))
gadm_netherlands = gpd.read_file(os.path.join('..', 'data', 'gadm', 'gadm36_NLD_2.shp'))
gadm_spain = gpd.read_file(os.path.join('..', 'data', 'gadm', 'gadm36_ESP_4.shp'))


In [7]:
file_gadm = os.path.join('..', 'data', 'gadm_table.csv')
gadm_info = pd.read_csv(file_gadm)

In [7]:
# respecting city fragmentation in France
gadm_france = gpd.read_file(os.path.join('..', 'data', 'gadm', 'gadm36_FRA_4_grouped_inc_lvl3.shp'))

## Preprocessing

### 1. Aligning GADM regions with NUTS LAU regions

#### 1.1. Data

In [8]:
# Used for mapping GADM regions to LOCAL ADMINISTRATIVE UNITS (LAU) and to get their Degree of Urbanisation (DEGURBA)
lau_eu = gpd.read_file(os.path.join('..', 'data', 'geometry', 'LAU_RG_01M_2020_3035.shp'))
gadm_5_france = gpd.read_file(os.path.join('..', 'data', 'gadm', 'gadm36_FRA_5.shp'))

In [9]:
columns_of_interest = ['LAU CODE', 'POPULATION', 'DEGURBA']

lau_fr_classification = pd.read_csv(os.path.join('..', 'metadata', 'FR-LAU-2019.csv'), sep=';')[columns_of_interest]
lau_es_classification = pd.read_csv(os.path.join('..', 'metadata', 'ES-LAU-2019.csv'), sep=';')[columns_of_interest]
lau_nl_classification = pd.read_csv(os.path.join('..', 'metadata', 'NL-LAU-2019.csv'), sep=';')[columns_of_interest]

lau_fr_classification.dropna(subset=['LAU CODE'], inplace=True)
lau_es_classification.dropna(subset=['LAU CODE'], inplace=True)
lau_nl_classification.dropna(subset=['LAU CODE'], inplace=True)

lau_es_classification['LAU CODE'] = lau_es_classification['LAU CODE'].astype(str)

2022-06-20 17:00:24,398 | WARNING : Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



In [18]:
lau_fr = lau_eu[lau_eu['CNTR_CODE'] == 'FR']
lau_es = lau_eu[lau_eu['CNTR_CODE'] == 'ES']
lau_nl = lau_eu[lau_eu['CNTR_CODE'] == 'NL']

len(gadm_5_france) - len(gadm_5_france.drop_duplicates(subset='NAME_5', keep=False))

len(gadm_5_france) - len(lau_fr)
len(gadm_spain) - len(lau_es)
len(gadm_netherlands) - len(lau_nl)

0

#### 1.2. Mapping of GADM / LU regions based on spatial proximity

In [12]:
def nearest_lau_region(df, lau_geom_col='geometry_lau', gadm_lvl=5):
    gadm = df['geometry'].iloc[0].centroid
    lau = df[lau_geom_col].centroid

    dis = lau.distance(gadm)

    if dis.min() > 500:
        logger.info(f"No matching LAU region found for {df[f'GID_{gadm_lvl}'].iloc[0]}. Centroid of nearest NUTS LAU region ({df.iloc[dis.argmin()]['LAU_ID']}) is {int(dis.min())}m.")

    df['centroid_distance'] = dis.min()
    return df.iloc[[dis.argmin()]]


def join_nearest_lau_region(gadm_df, lau_df, gadm_lvl=5):
    gadm_df = gadm_df.to_crs(3035)
    lau_df['geometry_lau'] = lau_df.geometry

    # join overlapping & touching LAU regions to GADM regions (results in duplicate GADM GID samples if region has multiple LAU neighbors)
    gadm_df_joined = gpd.sjoin_nearest(gadm_df, lau_df[['LAU_ID', 'LAU_NAME', 'geometry', 'geometry_lau']], how='left', max_distance=10000, distance_col='sjoin_distance')

    gadm_islands = gadm_df_joined[gadm_df_joined['sjoin_distance'] > 0][f'NAME_{gadm_lvl}'].unique()
    if len(gadm_islands) > 0:
        logger.warning(f'GADM regions found without any LAU neighbors: {gadm_islands}')

    # distinct vs. ambiguous := overlapping / touching GADM and LAU regions with same vs. different name
    gadm_df_lau_distinct = gadm_df_joined[gadm_df_joined[f'NAME_{gadm_lvl}'] == gadm_df_joined['LAU_NAME']]
    gadm_df_lau_distinct.drop_duplicates(f'GID_{gadm_lvl}', keep=False, inplace=True) # remove ambigiuous assignments, e.g. rare case that neighboring regions share same GADM lvl5 name
    gadm_df_lau_ambiguous = gadm_df_joined[~gadm_df_joined[f'GID_{gadm_lvl}'].isin(gadm_df_lau_distinct[f'GID_{gadm_lvl}'].unique())]

    # map nearest LAU region to ambiguous GADM regions
    gadm_df_lau_ambiguous_nearest = gadm_df_lau_ambiguous.groupby(f'GID_{gadm_lvl}', group_keys=False).apply(lambda group: nearest_lau_region(group, gadm_lvl=gadm_lvl))
    return pd.concat([gadm_df_lau_distinct, gadm_df_lau_ambiguous_nearest], axis=0)


def visualize_region_matching(m):
    %matplotlib qt
    _, ax = plt.subplots(1, 1)
    m.plot(ax=ax, column='LAU_NAME') #, color='blue')#, legend=True)
    gpd.GeoDataFrame(m, geometry=m['geometry_lau'], crs=3035).plot(ax=ax, column='LAU_NAME', hatch="//", alpha=0.2)


In [13]:
gadm_5_france_lau = join_nearest_lau_region(gadm_5_france, lau_fr, gadm_lvl=5)
gadm_spain_lau = join_nearest_lau_region(gadm_spain, lau_es, gadm_lvl=4)
gadm_netherlands_lau = join_nearest_lau_region(gadm_netherlands, lau_nl, gadm_lvl=2)

gadm_5_france_lau = gadm_5_france_lau.merge(lau_fr_classification, left_on='LAU_ID', right_on='LAU CODE', how='left')
gadm_spain_lau = gadm_spain_lau.merge(lau_es_classification, left_on='LAU_ID', right_on='LAU CODE', how='left')
gadm_netherlands_lau = gadm_netherlands_lau.merge(lau_nl_classification, left_on='LAU_ID', right_on='LAU CODE', how='left')

2022-06-20 17:01:34,882 | WARNING : 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2022-06-20 17:01:41,048 | WARNING : 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2022-06-20 17:01:41,114 | INFO : No matching LAU region found for FRA.1.1.1.2.16_1. Centroid of nearest NUTS LAU region (01286) is 896m.
2022-06-20 17:01:41,118 | INFO : No matching LAU region found for FRA.1.1.1.2.17_1. Centroid of nearest NUTS LAU region (01286) is 1499m.
2022-06-20 17:01:41,123 | INFO : No matching LAU region found for FRA.1.1.1.2.21_1. Centroid of nearest NUTS LAU region (01015) is 2049m.
2022-06-20 17:01:41,130 | INFO : No matchi

In [20]:
gadm_spain_lau.to_csv(os.path.join('..', 'metadata', 'ES-LAU-GADM-mapping.csv'), index=False)
gadm_netherlands_lau.to_csv(os.path.join('..', 'metadata', 'NL-LAU-GADM-mapping.csv'), index=False)
gadm_5_france_lau.to_csv(os.path.join('..', 'metadata', 'FR-LAU-GADM-mapping.csv'), index=False)
# gadm_spain_lau['DEGURBA'] 
# gadm_5_france_lau['DEGURBA']
# gadm_netherlands_lau['DEGURBA'] 


In [6]:
gadm_5_france_lau = pd.read_csv(os.path.join('..', 'metadata', 'FR-LAU-GADM-5-mapping.csv'))


#### 1.3. Special case France: aggregating GADM lvl 5 / LAU regions to GADM lvl 4 regions


Aggregation logic: choosing most common DEGURBA in region (for ties choose higher DEGURBA)

In [8]:

## proper aggregation using unique GID_4 instead of NAME_4
region_types = gadm_5_france_lau.groupby('GID_4')['DEGURBA'].agg(lambda x: pd.Series.mode(x).max()).to_dict()
gadm_france_lau = gadm_france.copy()
gadm_france_lau['DEGURBA'] = gadm_france['GID_4'].map(region_types)

# current workaround because fragmented city mappings are based on NAME not GID
# fragments_to_new_name = invert_fragmented_cities_mapping(fragmented_cities_lvl4['France'] + fragmented_cities_lvl3['France'])
# gadm_5_france_lau['NAME_4'] = gadm_5_france_lau['NAME_4'].map(fragments_to_new_name).fillna(gadm_5_france_lau['NAME_4'])

# region_types = gadm_5_france_lau.groupby('NAME_4')['DEGURBA'].agg(lambda x: pd.Series.mode(x).max()).to_dict()
# gadm_france_lau = gadm_france.copy()
# gadm_france_lau['DEGURBA'] = gadm_france['NAME_4'].map(region_types)

# if population data would become available for France with new LAU data release
# region_population = gadm_5_france_lau.groupby('GID_4')['POPULATION'].sum().to_dict()
# gadm_france['population'] = gadm_france['GID_4'].map(region_population)

gadm_france_lau.to_csv(os.path.join('..', 'metadata', 'FR-LAU-GADM-mapping.csv'), index=False)

#### 1.4. Validation

In [26]:
for country, m in {'spain': gadm_spain_lau, 'netherlands': gadm_netherlands_lau, 'france': gadm_5_france_lau}.items():
    m_avg = m.head(50)
    m_bad = m.sort_values('centroid_distance', ascending=False).head(10)
    print(f"{country}: share of assigments with distance > 1000m: {len(m[m['centroid_distance'] > 1000]) / len(m) * 100:.1f}%")
    visualize_region_matching(m_avg)
    visualize_region_matching(m_bad)


spain: share of assigments with distance > 1000m: 1.9%
netherlands: share of assigments with distance > 1000m: 35.6%
france: share of assigments with distance > 1000m: 6.0%


In [341]:

_, ax = plt.subplots(1, 1)
lau_fr_ambiguous[lau_fr_ambiguous['LAU_ID'].isin(['01015', '01061', '73028'])].plot(ax=ax, column='LAU_NAME', cmap='tab20', legend=True)
gadm_5_france_ambiguous[gadm_5_france_ambiguous['GID_5'] == 'FRA.1.1.1.2.6_1'].to_crs(3035).plot(ax=ax, color='red', alpha=0.5)

# lau_fr[lau_fr['LAU_ID'].isin(['08103'])].plot(ax=ax, column='LAU_NAME', cmap='tab20', legend=True)
# gadm_5_france_ambiguous[gadm_5_france_ambiguous['GID_5'] == 'FRA.6.1.4.1.2_1'].to_crs(3035).plot(ax=ax, color='red', alpha=0.5)

2022-05-09 09:23:10,766 | WARNING : The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.



<AxesSubplot:>

### 2. Grouping of fragmented cities (France)

In [419]:
import json

with open(os.path.join('..', 'data', 'fragmented-cities-candidates-regex-level-3.json'), 'r') as file:
    fragmented_cities_lvl3 = json.load(file)

with open(os.path.join('..', 'data', 'fragmented-cities-candidates-regex-level-4.json'), 'r') as file:
    fragmented_cities_lvl4 = json.load(file)

In [22]:
def invert_fragmented_cities_mapping(l):
    new_dic = {}
    for region in l:
        new_name = region[0]
        fragments = region[1]
        for x in fragments:
            new_dic[x] = new_name
    return new_dic

In [421]:
fragments_to_new_name = invert_fragmented_cities_mapping(fragmented_cities_lvl4['France'] + fragmented_cities_lvl3['France'])
overview_france['city'] = overview_france['city'].map(fragments_to_new_name).fillna(overview_france['city'])
grouped_overview_france = overview_france.groupby('city')[['bldgs_n_tot', 'a_tot']].sum()
grouped_overview_france = pd.concat([grouped_overview_france, overview_france.groupby('city')['age_p', 'age_p_1901_1950', 'age_p_1951_2000', 'age_p_2001_2022', 'height_mean'].mean()], axis=1)
grouped_overview_france.reset_index(inplace=True)

2022-05-09 10:58:21,270 | WARNING : Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



### 3. Handling of duplicate city names (Spain)

In [27]:
# logic from https://github.com/ai4up/eubucco/blob/main/eubucco/preproc/db_set_up.py#L253-L262

# add region_name to city name where we have duplicates
g_d = gadm_spain_lau[gadm_spain_lau.duplicated(subset=['NAME_4'])]
g_d['NAME_4'] = g_d['NAME_4'] + ' (' + g_d['NAME_1'] + ')'

# in cases where we have more than one duplicated; add orignal index as str behind dupl
g_d_2 = g_d[g_d.duplicated(subset=['NAME_4'])]
g_d = g_d.drop(g_d_2.index)

g_d_2['NAME_4'] = g_d_2['NAME_4'] + '_' + g_d_2.index.map(str)
g_d = g_d.append(g_d_2)

gadm_spain_lau = gadm_spain_lau.drop(g_d.index).append(g_d)


2022-06-20 17:18:53,498 | WARNING : 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2022-06-20 17:18:53,519 | WARNING : The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

2022-06-20 17:18:53,541 | WARNING : The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 4. Calculation of additional selection metrics

In [29]:
country_dfs = {}
country_infos = [
    ('netherlands', gadm_netherlands_lau, overview_netherlands),
    ('france', gadm_france_lau, grouped_overview_france),
    ('spain', gadm_spain_lau, overview_spain),
]

for country, gadm, overview in country_infos:
    gadm_level = gadm_info[gadm_info['country_name'] == country]['level_city'].values[0]

    df = gadm[[f'NAME_{gadm_level}', f'NAME_{gadm_level - 1}', 'DEGURBA', 'geometry']].merge(overview, left_on=f'NAME_{gadm_level}', right_on='city', how='inner')

    df['a_3d'] = df['height_mean'] * df['a_tot']
    df['bldgs_per_km2'] = df['bldgs_n_tot'] / df['geometry'].to_crs(3035).area * 1000**2
    df['build_up_area'] = df['a_tot'] / df['geometry'].to_crs(3035).area
    df['build_up_area_3d'] = df['a_3d'] / df['geometry'].to_crs(3035).area
    df[['age_p_1901_1950', 'age_p_1951_2000', 'age_p_2001_2022']] = df[['age_p_1901_1950', 'age_p_1951_2000', 'age_p_2001_2022']].div(df['age_p'], axis=0)
    country_dfs[country] = df



## Selection

### 1. Inspection

In [31]:
country_dfs['france']['DEGURBA'].value_counts()

3.0    2706
2.0     460
1.0     213
Name: DEGURBA, dtype: int64

In [9]:
# cities have fewer buildings / building to inhabitants ratio is lower (maybe because of more coarse building block information & different gadm boundaries)
# * have different rural, town, urban criterias for Spain
# * use inhabitant information instead of buildings to differentiate between city types
# * just use n largest cities per country
# country_dfs['netherlands'].sort_values(by='bldgs_n_tot', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area', 'a_tot']].head(50)
country_dfs['netherlands'][country_dfs['netherlands']['city'].isin(['Maastricht', 'Almelo', 'Ede'])].sort_values(by='bldgs_n_tot', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area', 'a_tot']]
# country_dfs['france'][country_dfs['france']['city'].isin(['Aix-en-Provence', 'Rennes', 'Limoges'])].sort_values(by='bldgs_n_tot', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area', 'a_tot']]
# country_dfs['spain'][country_dfs['spain']['city'].isin(['Valencia', 'Cartagena', 'Vigo', 'Zaragoza', 'Alicante', 'Oviedo', 'Málaga', 'Bilbao', 'Palma de Mallorca'])].sort_values(by='bldgs_n_tot', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area']]

NameError: name 'country_dfs' is not defined

In [472]:
towns(country_dfs['spain']).sort_values(by='build_up_area', ascending=False)[['city', 'DEGURBA', 'a_3d', 'age_p', 'a_tot', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area', 'height_mean', 'build_up_area_3d']].head(10)

,city,DEGURBA,a_3d,age_p,a_tot,bldgs_n_tot,bldgs_per_km2,build_up_area,height_mean,build_up_area_3d
5877,Emperador,2.0,6.514620e+04,1.0,9364.123,46.0,1999.792201,0.407093,6.957,2.832149
5901,Beniparrell,2.0,4.151645e+06,1.0,878840.975,1158.0,306.730383,0.232787,4.724,1.099685
5918,Benirredrà,2.0,3.899088e+05,1.0,63081.829,398.0,1104.335226,0.175034,6.181,1.081884
8230,Barañain,2.0,3.189699e+06,0.0,240351.044,175.0,124.616550,0.171153,13.271,2.271367
7472,Rafelbuñol,2.0,3.242501e+06,1.0,599796.658,2492.0,610.512173,0.146943,5.406,0.794376
7462,L'Eliana,2.0,4.658844e+06,1.0,1254063.019,9096.0,1033.306949,0.142462,3.715,0.529245
5972,Almussafes,2.0,8.837991e+06,1.0,1508447.061,2145.0,198.803237,0.139806,5.859,0.819124
5792,Almazora,2.0,2.253938e+07,1.0,4534173.427,9824.0,297.887106,0.137487,4.971,0.683448
5917,Beniflá,2.0,4.015334e+05,1.0,83670.220,206.0,334.869280,0.136013,4.799,0.652724
1406,Reinosa,2.0,4.175792e+06,1.0,584926.748,1619.0,374.064444,0.135145,7.139,0.964803


In [473]:
rural_area(country_dfs['spain']).sort_values(by='bldgs_per_km2', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area']].head(10)

,city,age_p,bldgs_n_tot,bldgs_per_km2,build_up_area
1357,Maleján,0.99,168.0,5492.373429,0.416983
7491,La Aldea del Obispo,1.00,602.0,1815.811882,0.149351
7737,El Puente del Arzobispo,0.99,1147.0,1115.135712,0.140588
5748,Geldo,1.00,500.0,970.415946,0.068301
4751,Castellfollit de la Roca,1.00,419.0,655.816987,0.089761
7540,A Illa de Arousa,0.99,2430.0,380.185520,0.044974
659,Carrión de los Céspedes,1.00,2216.0,364.362039,0.042932
5855,Llocnou d'En Fenollet,1.00,519.0,362.255458,0.046513
4864,Breda,1.00,1763.0,346.729071,0.058739
4868,Hostalric,1.00,1124.0,328.174565,0.087859


In [496]:
towns(country_dfs['france']).sort_values(by='bldgs_n_tot', ascending=False)[['city', 'age_p', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area']].head(10)

,city,age_p,bldgs_n_tot,bldgs_per_km2,build_up_area
2491,Léguevin,0.370000,49985.0,380.588926,0.031374
2639,Perpignan,0.560000,49584.0,718.442325,0.108409
887,Chartres,0.545000,49248.0,130.743933,0.016466
2116,Castelnau-de-Médoc,0.460000,49195.0,51.041416,0.005028
2144,Limoges,0.610000,48842.0,600.196426,0.093319
1139,Bischwiller,0.490000,48643.0,270.102403,0.022232
2351,Narbonne,0.483333,48007.0,164.940975,0.017576
1484,Seclin,0.550000,47961.0,501.861421,0.063663
2468,Muret,0.400000,47915.0,283.072550,0.027908
1450,Cambrai,0.576667,47761.0,274.255922,0.031340


### 2. Criterias for geographic & urban typology diversity

In [33]:
import warnings
warnings.filterwarnings('ignore', message='Geometry is in a geographic CRS')

centroid_france_x = 2.2137
centroid_france_y = 46.2276
centroid_netherlands_x = 5.2913
centroid_netherlands_y = 52.1326
centroid_spain_x = -3.7037
centroid_spain_y = 40.4165


centroid_coordinates = {
    'france': (centroid_france_x, centroid_france_y),
    'netherlands': (centroid_netherlands_x, centroid_netherlands_y),
    'spain': (centroid_spain_x, centroid_spain_y),
}


def north_eastern(df, centroid_x, centroid_y):
    city_centroids = df['geometry'].to_crs(4326).centroid
    north_eastern_mask = (city_centroids.x > centroid_x) & (city_centroids.y > centroid_y)
    return df[north_eastern_mask]

def south_eastern(df, centroid_x, centroid_y):
    city_centroids = df['geometry'].to_crs(4326).centroid
    south_eastern_mask = (city_centroids.x > centroid_x) & (city_centroids.y < centroid_y)
    return df[south_eastern_mask]

def north_western(df, centroid_x, centroid_y):
    city_centroids = df['geometry'].to_crs(4326).centroid
    north_western_mask = (city_centroids.x < centroid_x) & (city_centroids.y > centroid_y)
    return df[north_western_mask]

def south_western(df, centroid_x, centroid_y):
    city_centroids = df['geometry'].to_crs(4326).centroid
    south_western_mask = (city_centroids.x < centroid_x) & (city_centroids.y < centroid_y)
    return df[south_western_mask]


In [34]:
def urban_centers(df):
    return df[df['DEGURBA'] == 1]

def towns(df):
    return df[df['DEGURBA'] == 2]

def rural_area(df):
    return df[df['DEGURBA'] == 3]

In [830]:
for country, df in country_dfs.items():
    print(country)
    print(df.nlargest(50, ['a_tot'])[['bldgs_n_tot', 'bldgs_per_km2', 'a_tot', 'build_up_area']].quantile(0.25))
    # print(df[['a_3d', 'build_up_area_3d', 'a_tot']].mean())

netherlands
bldgs_n_tot      3.279200e+04
bldgs_per_km2    3.125520e+02
a_tot            5.470344e+06
build_up_area    5.355809e-02
Name: 0.25, dtype: float64
france
bldgs_n_tot      4.697925e+04
bldgs_per_km2    2.433429e+02
a_tot            7.021310e+06
build_up_area    3.496673e-02
Name: 0.25, dtype: float64
spain
bldgs_n_tot      1.932050e+04
bldgs_per_km2    1.081154e+02
a_tot            6.324644e+06
build_up_area    2.798800e-02
Name: 0.25, dtype: float64


### 3. Criteria for differentiating between rural and suburban areas

In [468]:
import webbrowser
import random

def filter_suburban_areas(settlements, urban_centers, verbose=False):
    urban_center_centroids = urban_centers['geometry'].to_crs(4326).centroid
    city_centroids = settlements['geometry'].to_crs(4326).centroid

    matrix = geometry.pairwise_distance_matrix(city_centroids, urban_center_centroids)
    matrix = matrix - city_radius(urban_centers)
    matrix = matrix - city_radius(settlements).T

    min_km_to_nearest_city = 10
    autonomous_settlements = [city_idx for city_idx, nearby_urban_centers in enumerate(matrix) if nearby_urban_centers.min() > min_km_to_nearest_city]

    if verbose:
        suburban_settlements = [(settlements.iloc[city_idx]['city'], urban_centers.iloc[nearby_urban_centers.argmin()]['city']) for city_idx, nearby_urban_centers in enumerate(matrix) if nearby_urban_centers.min() < min_km_to_nearest_city]
        print(f'The following suburban {len(suburban_settlements)} areas were identified and removed:', *suburban_settlements, sep='\n- ')

        for start, destination in random.choices(suburban_settlements, k=5):
            webbrowser.open(f'https://www.google.com/maps/dir/{start}/{destination}/')


    return settlements.iloc[autonomous_settlements]


def city_radius(df):
    # defined as shortest distance between a city's centroid and its gadm boundary in km
    return np.array([df['geometry'].boundary.to_crs(3035).distance(df['geometry'].to_crs(3035).centroid) / 1000])


# Exemplary filtering
# df = country_dfs['france']
# urban_regions = df[(df['bldgs_per_km2'] > 500) & (df['bldgs_n_tot'] > 10000)]
# rural_regions = df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'].between(0, 200)) & (df['bldgs_n_tot'] < 10000)]

# filter_suburban_areas(rural_regions, urban_regions, verbose=True)
# filter_suburban_areas(towns(df), urban_regions, verbose=True)

# urban_regions[['city', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area']].sort_values('bldgs_n_tot', ascending=False)
# rural_regions[['city', 'bldgs_n_tot', 'bldgs_per_km2', 'build_up_area']].sort_values('bldgs_per_km2', ascending=False)

### 4. Criteria for outlier removal

In [37]:
def inliers(df, low=0.1, high=0.8, iqr_tolerance=False):
    quantile_cols = ['age_p_1901_1950', 'age_p_1951_2000', 'age_p_2001_2022']
    quantiles = df[quantile_cols].quantile([low, high])
    quantiles.loc['IQR'] = quantiles.loc[high] - quantiles.loc[low]

    if iqr_tolerance:
        quantiles.loc[high] = quantiles.loc[high] + quantiles.loc['IQR']
        quantiles.loc[low] = quantiles.loc[low] - quantiles.loc['IQR']


    for col in quantile_cols:
        df = df[(df[col] > quantiles.loc[low, col]) & (df[col] < quantiles.loc[high, col])]

    return df

### 5. Final selection of cities (with respective # of buildings with age attribute)

['Borger-Odoorn',
 'Weststellingwerf',
 'Dinkelland',
 'Het Bildt',
 'Buren',
 'Roggel en Neer',
 'Drechterland',
 'Veere',
 'Goedereede',
 'Hérisson',
 'Huriel',
 'Le Montet',
 'Souvigny',
 'Thonon-les-Bains-Ouest',
 'Montigny-sur-Aube',
 'Recey-sur-Ource',
 'Marchaux',
 'Saint-Hippolyte',
 'Vercel-Villedieu-le-Camp',
 'Adelans-et-le-Val-de-Bithaine',
 'Champagney',
 'Héricourt-Ouest',
 'Villers-Farlay',
 'Arinthod',
 'Orgelet',
 'Sellières',
 'Château-Chinon (Ville)',
 'Tannay',
 'Pougues-les-Eaux',
 'Chalon-sur-Saône-Sud',
 "La Chapelle-d'Angillon",
 'Lorris',
 'Pithiviers',
 'Estissac',
 'Lusigny-sur-Barse',
 'Wasselonne',
 'Obernai',
 'Altkirch',
 'Hirsingue',
 'Wintzenheim',
 'Guebwiller',
 'Ribeauvillé',
 'Clefmont',
 'Fayl-Billot',
 'Saint-Dizier-Nord-Est',
 'Longuyon',
 'Toul-Nord',
 'Château-Salins',
 'Phalsbourg',
 'Réchicourt-le-Château',
 'Sarralbe',
 'Metzervisse',
 'Laon-Nord',
 'Tergnier',
 'Soissons-Nord',
 'Cambrai-Ouest',
 'Hazebrouck-Sud',
 'Saint-Amand-les-Eaux-Riv

In [110]:
for country, df in country_dfs.items():
    print(f'{len(df) - len(df.dropna(subset=["DEGURBA"]))} unknown DEGURBA types for {country}.')
    df['DEGURBA'].fillna('unknown', inplace=True)

0 unknown DEGURBA types for netherlands.
0 unknown DEGURBA types for france.
0 unknown DEGURBA types for spain.


In [124]:

with open(os.path.join('..', 'metadata', 'hyperparameter-tuning-cities.json'), 'r', encoding='utf-8') as f:
    selected_cities = json.load(f)
utils.flatten([region_cities for type_cities in selected_cities['netherlands'].values() for region_cities in type_cities.values()])
# selected_cities['netherlands'].values()

['Apeldoorn',
 'Bathmen',
 'Brunssum',
 'Veenendaal',
 'Amsterdam',
 'Bloemendaal',
 'Heemstede',
 'IJsselstein',
 'Vlissingen',
 'Alphen aan den Rijn',
 'Binnenmaas',
 'Dronten',
 'Tytsjerksteradiel',
 'Voorst',
 'Oldenzaal',
 'Staphorst',
 'Bunschoten',
 'Beuningen',
 'Renkum',
 'Deurne',
 'Valkenswaard',
 'Dinxperlo',
 'Groesbeek',
 'Rijnwaarden',
 'Nuth',
 'Maasdonk',
 'Sint-Oedenrode',
 'Blaricum',
 'Uithoorn',
 'De Ronde Venen',
 'Jacobswoude',
 'Zijpe',
 'Drimmelen',
 'Geertruidenberg',
 'Woerden',
 'Kapelle',
 'Brielle',
 'Vianen',
 'Liesveld',
 'Borger-Odoorn',
 'Weststellingwerf',
 'Dinkelland',
 'Het Bildt',
 'Buren',
 'Roggel en Neer',
 'Drechterland',
 'Veere',
 'Goedereede']

In [113]:
import collections
import json
import pickle

selected_cities = collections.defaultdict(lambda: collections.defaultdict(dict))

regions = {'north-east': north_eastern, 'south-east': south_eastern, 'north-west': north_western, 'south-west': south_western}
deburba_types = {1: 'urban', 2: 'town', 3: 'rural', 'unknown': 'unknown'}

for country, df in country_dfs.items():
    # df = df[df['age_p'] > 0.3]

    for type, df_type in df.groupby('DEGURBA'):

        for region, filter in regions.items():
            df_region = filter(df_type, *centroid_coordinates[country])

            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # df_region_sample = utils.sample_cities_with_distributional_constraint(df_region, frac=0.2, attr='a_3d', min_samples_per_group=5)
                if len(df_region) > 0:
                    df_region_sample = utils.sample_cities(df_region, frac=0.1)
                    selected_cities[country][type][region] = list(df_region_sample['city'].values)

                    print(f'{country} - {deburba_types[type]} area - {region} region: {(len(df_region_sample) / len(df)) * 100:.1f}% (#cities {len(df_region_sample)}) (#buildings {str(int((df_region_sample["bldgs_n_tot"] * df_region_sample["age_p"]).sum()))[:-3]}k)')


with open(os.path.join('..', 'metadata', f'selected-cities.json'), 'w', encoding='utf-8') as f:
    json.dump(selected_cities, f, ensure_ascii=False, indent=4)

with open(os.path.join('..', 'metadata', f'selected-cities.pkl'), 'wb') as f:
    selected_cities_flattened = utils.flatten([region_cities for country_cities in selected_cities.values() for type_cities in country_cities.values() for region_cities in type_cities.values()])
    print(f'Overall selected {len(selected_cities_flattened)} cities')
    pickle.dump(selected_cities_flattened, f)

    # print(f'{country}: share of region {region}: {(len(filter(df)) / len(df)):.3f}')

# n_type = {'urban': 3, 'town': 30, 'rural': 30}
# settlement_types = {'urban': urban_centers, 'town': towns, 'rural': rural_area}

# for country, df in country_dfs.items():
#     for type in settlement_types:
#         df_type = settlement_types[type](df)

#         # special cases
#         if country == 'france' and type == 'rural':
#             # gadm level 4 yields partially fragmented cities;
#             # therefore, rural settlements are checked to not be a suburban area / have a minimum distance to nearby urban regions
#             urban_regions = df[(df['bldgs_per_km2'] > 500) & (df['bldgs_n_tot'] > 10000)]
#             df_type = filter_suburban_areas(df_type, urban_regions)

#         # if type != 'urban':
#         #     df_type = inliers(df_type, low=0.1, high=0.9)

#         for region in regions:
#             df_cat = regions[region](df_type, *centroid_coordinates[country])

#             # n = int(len(df_cat) / len(regions.keys()))
#             # if len(df_cat) > 0:
#             #     df_cat = df_cat.sample(n=min(n_type[type], len(df_cat)), weights='age_p', random_state=dataset.GLOBAL_REPRODUCIBILITY_SEED)

#             print(f'{country}: share of {type} areas in region {region}: {(len(df_cat) / len(df)) * 100:.1f}% (#{type} {len(df_cat)}) (#buildings {str(int((df_cat["bldgs_n_tot"] * df_cat["age_p"]).sum()))[:-3]}k)')
#             selected_cities[type][region] = list(df_cat['city'].values)


#     with open(os.path.join('..', 'data', f'{country}-selected-cities-new.json'), 'w', encoding='utf-8') as f:
#         json.dump(selected_cities, f, ensure_ascii=False, indent=4)


netherlands - urban area - north-east region: 0.4% (#cities 2) (#buildings 108k)
netherlands - urban area - south-east region: 0.4% (#cities 2) (#buildings 44k)
netherlands - urban area - north-west region: 0.6% (#cities 3) (#buildings 195k)
netherlands - urban area - south-west region: 0.8% (#cities 4) (#buildings 93k)
netherlands - town area - north-east region: 1.2% (#cities 6) (#buildings 110k)
netherlands - town area - south-east region: 2.0% (#cities 10) (#buildings 141k)
netherlands - town area - north-west region: 1.0% (#cities 5) (#buildings 58k)
netherlands - town area - south-west region: 1.4% (#cities 7) (#buildings 105k)
netherlands - rural area - north-east region: 0.8% (#cities 4) (#buildings 72k)
netherlands - rural area - south-east region: 0.4% (#cities 2) (#buildings 24k)
netherlands - rural area - north-west region: 0.2% (#cities 1) (#buildings 6k)
netherlands - rural area - south-west region: 0.4% (#cities 2) (#buildings 33k)
france - urban area - north-east region

In [58]:
country_dfs['france']['age_p'].hist(bins=7)

<AxesSubplot:>

### X. Deprecated approaches

#### Custom urban typology classifcation

In [ ]:
# Netherlands
# def urban_centers(df):
#     return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'] > 500) & (df['bldgs_n_tot'] > 75000)]

# def towns(df):
#     return df[(df['age_p'] > 0.3) & df['bldgs_per_km2'].between(200, 500) & df['bldgs_n_tot'].between(10000, 50000)]

# def rural_area(df):
#     return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'].between(0, 200)) & (df['bldgs_n_tot'] < 10000)]

# France
# def urban_centers(df):
#     return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'] > 1000) & (df['build_up_area'] > 0.1) & (df['bldgs_n_tot'] > 50000)]

# def towns(df):
#     return df[(df['age_p'] > 0.3) & df['bldgs_per_km2'].between(100, 500) & df['bldgs_n_tot'].between(20000, 50000)]

# def rural_area(df):
#     return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'].between(0, 50)) & (df['bldgs_n_tot'] < 10000)]


# All countries
# def urban_centers(df):
#     return df[(df['age_p'] > 0.3) & (df['height_mean'] > 4) & (df['a_tot'] > 5000000)]
#     # return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'] > 500) & (df['bldgs_n_tot'] > 50000)]

# def towns(df):
#     return df[(df['age_p'] > 0.3) & df['bldgs_per_km2'].between(200, 500) & df['bldgs_n_tot'].between(10000, 50000)]

# def rural_area(df):
#     return df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'].between(0, 200)) & (df['bldgs_n_tot'] < 10000)]

# def rural_area_france(df):
#     rural_regions = df[(df['age_p'] > 0.3) & (df['bldgs_per_km2'].between(0, 200)) & (df['bldgs_n_tot'] < 10000)]
#     urban_regions = df[(df['bldgs_per_km2'] > 500) & (df['bldgs_n_tot'] > 10000)]
#     return filter_suburban_areas(rural_regions, urban_regions)

def largest_cities(df):
    c1 = df.nlargest(50, ['bldgs_n_tot'])['city']
    c2 = df.nlargest(50, ['a_tot'])['city']
    # c3 = df.nlargest(50, ['build_up_area'])['city']
    cities = set(c1).intersection(c2)
    return  df[df['city'].isin(cities) & (df['age_p'] > 0.3)]

def relative_urban_centers(df):
    quantiles = df.nlargest(50, ['a_tot'])[['bldgs_n_tot', 'bldgs_per_km2', 'a_tot', 'build_up_area']].quantile(0.25)
    return df[
        (df['age_p'] > 0.3) &
        (df['bldgs_n_tot'] > quantiles['bldgs_n_tot']) &
        # (df['bldgs_per_km2'] > quantiles['bldgs_per_km2']) &
        (df['a_tot'] > quantiles['a_tot'])
        ]

def urban_centers(df):
    return df[(df['age_p'] > 0.3) & (df['a_3d'] > 40000000) & (df['build_up_area_3d'] > 0.3)].sort_values(by='a_3d', ascending=False)

def rural_area(df):
    return df[(df['age_p'] > 0.3) & (df['a_3d'] < 5000000) | (df['build_up_area_3d'] < 0.03)].sort_values(by='a_3d', ascending=False)

def towns(df):
    return df.drop(list(urban_centers(df).index) + list(rural_area(df).index))

#### Regional classification based on K-means clustering

In [87]:
from sklearn.cluster import KMeans
import numpy as np
import warnings

warnings.filterwarnings('ignore', message='Explicit initial center position passed')


def regional_clustering(df):
    city_centroids = df['geometry'].to_crs(4326).centroid
    df['lat'] = city_centroids.y
    df['lon'] = city_centroids.x

    nswe_inital_clusters = np.array([[52.2451, 4.8433], [52.63445462289726, 6.336618136624664], [51.5842, 4.1645], [51.4580, 5.8240]])
    kmeans = KMeans(n_clusters=4, random_state=0, init=nswe_inital_clusters)
    kmeans.fit(df[['lat', 'lon']])

    kmeans.cluster_centers_
    # city_region = dict(zip(df['city'], kmeans.labels_))
    return kmeans.labels_


def north_eastern(df):
    labels = regional_clustering(df)
    return df[labels == 1]

def south_eastern(df):
    labels = regional_clustering(df)
    return df[labels == 3]

def north_western(df):
    labels = regional_clustering(df)
    return df[labels == 0]

def south_western(df):
    labels = regional_clustering(df)
    return df[labels == 2]



In [510]:


len(set(country_dfs['spain'].sort_values(by='bldgs_n_tot', ascending=False).city.unique()[:50]) - set(pd.read_csv(os.path.join('/Users/D062794/Downloads', 'csvData.csv')).name.unique()))
country_dfs['spain'].sort_values(by='bldgs_n_tot', ascending=False).city.unique()[:50]

'Badajoz' in pd.read_csv(os.path.join('/Users/D062794/Downloads', 'csvData.csv')).name.unique()


False

#### Alterative approaches to GADM LAU alignment

In [ ]:
# alternative straight-forward approach based on name merging
lau_es = lau_eu[lau_eu['CNTR_CODE'] == 'ES']
gadm_spain.merge(lau_es[['LAU_ID', 'LAU_NAME']], left_on='NAME_4', right_on='LAU_NAME', how='inner')

In [42]:
# alternative approach
# first match based on name and sjoin_nearest only ambiguous regions (to reduce computation complexity of sjoining all regions)

# drop all duplicates from both dfs and align them later with geopandas.sjoin_nearest
lau_fr = lau_eu[lau_eu['CNTR_CODE'] == 'FR']
lau_fr_distinct = lau_fr.drop_duplicates(subset='LAU_NAME', keep=False)
gadm_5_france_distinct = gadm_5_france.drop_duplicates(subset='NAME_5', keep=False)

gadm_france_lau = gadm_5_france_distinct.merge(lau_fr_distinct[['LAU_ID', 'LAU_NAME']], left_on='NAME_5', right_on='LAU_NAME', how='left')

lau_fr_ambiguous = lau_fr[~lau_fr['LAU_ID'].isin(gadm_france_lau['LAU_ID'])]
gadm_5_france_ambiguous = gadm_5_france.drop(gadm_5_france_distinct.index)

gadm_5_france_ambiguous_joined = gpd.sjoin_nearest(gadm_5_france_ambiguous.to_crs(3035), lau_fr_ambiguous[['LAU_ID', 'LAU_NAME', 'geometry']], how='left', max_distance=10000, distance_col='distance')
# https://stackoverflow.com/questions/71434825/attributeerror-module-geopandas-has-no-attribute-sjoin-nearest
# gadm_france_lau[gadm_france_lau.duplicated('GID_5')]
# gadm_france_lau[gadm_france_lau['GID_5'] == 'FRA.1.1.1.2.6_1']

In [198]:
# gadm_5_france[~gadm_5_france['GID_5'].isin(gadm_5_france_lau['GID_5'].unique())]
# _, ax = plt.subplots(1, 1)
# gadm_5_france_lau_distinct[gadm_5_france_lau_distinct['GID_5'].duplicated(keep=False)]
# dup = gadm_5_france_lau_distinct[gadm_5_france_lau_distinct['LAU_NAME'] == 'Seyssel']
# dup.plot(column='GID_5', ax=ax)
# gpd.GeoDataFrame(dup, geometry=dup['geometry_lau'], crs=3035).plot(ax=ax, color='red', hatch="//", alpha=0.1)

len(gadm_5_france) == len(gadm_5_france_lau)


True